In [1]:

import json
import numpy as np
import evaluate
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
# import torch.nn as nn
from datasets import load_dataset, Dataset
import torch
import time
# from peft import PeftModel, PeftConfig
import phoenix as px
from phoenix.otel import register

SyntaxError: multiple exception types must be parenthesized (__init__.py, line 56)

In [3]:


device = "mps"


In [6]:

# model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model_id = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="right")
if not tokenizer.pad_token:
    tokenizer.pad_token = tokenizer.eos_token
# tokenizer.chat_template = '{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- if strftime_now is defined %}\n        {%- set date_string = strftime_now("%d %b %Y") %}\n    {%- else %}\n        {%- set date_string = "26 Jul 2024" %}\n    {%- endif %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "" %}\n{%- endif %}\n\n{#- System message #}\n{{- "<|start_header_id|>system<|end_header_id|>\\n\\n" }}\n{%- if tools is not none %}\n    {{- "Environment: ipython\\n" }}\n{%- endif %}\n{%- if tools is not none and not tools_in_user_message %}\n    {{- "You have access to the following functions. To call a function, please respond with JSON for a function call." }}\n    {{- \'Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.\' }}\n    {{- "Do not use variables.\\n\\n" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- "\\n\\n" }}\n    {%- endfor %}\n{%- endif %}\n{{- system_message }}\n{{- "<|eot_id|>" }}\n\n{#- Custom tools are passed in a user message with some extra guidance #}\n{%- if tools_in_user_message and not tools is none %}\n    {#- Extract the first user message so we can plug it in here #}\n    {%- if messages | length != 0 %}\n        {%- set first_user_message = messages[0][\'content\']|trim %}\n        {%- set messages = messages[1:] %}\n    {%- else %}\n        {{- raise_exception("Cannot put tools in the first user message when there\'s no first user message!") }}\n{%- endif %}\n    {{- \'<|start_header_id|>user<|end_header_id|>\\n\\n\' -}}\n    {{- "Given the following functions, please respond with a JSON for a function call " }}\n    {{- "with its proper arguments that best answers the given prompt.\\n\\n" }}\n    {{- \'Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.\' }}\n    {{- "Do not use variables.\\n\\n" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- "\\n\\n" }}\n    {%- endfor %}\n    {{- first_user_message + "<|eot_id|>"}}\n{%- endif %}\n\n{%- for message in messages %}\n    {%- if not (message.role == \'ipython\' or message.role == \'tool\' or \'tool_calls\' in message) %}\n        {{- \'<|start_header_id|>\' + message[\'role\'] + \'<|end_header_id|>\\n\\n\'+ message[\'content\'] | trim + \'<|eot_id|>\' }}\n    {%- elif \'tool_calls\' in message %}\n        {%- if not message.tool_calls|length == 1 %}\n            {{- raise_exception("This model only supports single tool-calls at once!") }}\n        {%- endif %}\n        {%- set tool_call = message.tool_calls[0].function %}\n        {{- \'<|start_header_id|>assistant<|end_header_id|>\\n\\n\' -}}\n        {{- \'{"name": "\' + tool_call.name + \'", \' }}\n        {{- \'"parameters": \' }}\n        {{- tool_call.arguments | tojson }}\n        {{- "}" }}\n        {{- "<|eot_id|>" }}\n    {%- elif message.role == "tool" or message.role == "ipython" %}\n        {{- "<|start_header_id|>ipython<|end_header_id|>\\n\\n" }}\n        {%- if message.content is mapping or message.content is iterable %}\n            {{- message.content | tojson }}\n        {%- else %}\n            {{- message.content }}\n        {%- endif %}\n        {{- "<|eot_id|>" }}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- \'<|start_header_id|>assistant<|end_header_id|>\\n\\n\' }}\n{%- endif %}\n'

model = AutoModelForCausalLM.from_pretrained("/Users/mann.kurani.ctr/Desktop/Personal_Code/SQL Finetune/models/finetuned_final_2_tinyllama").to(device)
# model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
# model = PeftModel.from_pretrained(model, "/Users/mann.kurani.ctr/Desktop/Personal_Code/SQL Finetune/models/finetuned_final_2_tinyllama").to(device)


In [4]:

# lora_config = LoraConfig(
#     task_type="CAUSAL_LM",
#     r = 16,
#     lora_alpha=32,
#     lora_dropout=0.1,
#     target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
# )

# model = get_peft_model(model, lora_config)
# model.print_trainable_parameters()


In [7]:


def dataset_creation(n: int) -> Dataset:
    """Loads and trims the text-to-SQL dataset."""
    dataset = load_dataset("gretelai/synthetic_text_to_sql")
    columns_to_keep = ['sql_prompt', 'sql_context', 'sql']

    # Filter only required columns
    dataset = dataset["train"].remove_columns(
        [col for col in dataset["train"].column_names if col not in columns_to_keep]
    )

    return dataset.select(range(n))



In [8]:

def tokenize_fn(batch):
    prompts = []
    sql_outputs = []

    for sql_prompt, sql_context, sql in zip(batch["sql_prompt"], batch["sql_context"], batch["sql"]):
        # Build the prompt using chat format
        prompt = tokenizer.apply_chat_template(
            [
                {"role": "system", "content": "You are a helpful assistant that converts SQL prompts and context into SQL queries. Do not give any additional comments or anything other than the sql query itself."},
                {"role": "user", "content": json.dumps({
                    "sql_prompt": sql_prompt,
                    "sql_context": sql_context
                })}
            ],
            tokenize=False,
            add_generation_prompt=True
        )
        prompts.append(prompt)

        # Format SQL output with EOS
        sql_output = f'[{{"sql": "{sql.strip()}"}}]{tokenizer.eos_token}'
        sql_outputs.append(sql_output)

    # Full sequence = prompt + sql_output
    full_texts = [p + o for p, o in zip(prompts, sql_outputs)]

    # Tokenize full sequences
    full_encodings = tokenizer(
        full_texts,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt"
    )

    input_ids = full_encodings["input_ids"]
    attention_mask = full_encodings["attention_mask"]

    # Get prompt lengths to mask label tokens for prompts
    prompt_lengths = [
        len(tokenizer(p, truncation=True, max_length=512)["input_ids"])
        for p in prompts
    ]

    labels = input_ids.clone()
    for i, prompt_len in enumerate(prompt_lengths):
        labels[i, :prompt_len] = -100  # Mask prompt tokens

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }


In [9]:


df = dataset_creation(5000).train_test_split(0.01)


train_df = df["train"]
eval_df = df["test"]


train_df = train_df.map(tokenize_fn, batched=True)
eval_df = eval_df.map(tokenize_fn, batched=True)

Map:   0%|          | 0/4950 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [8]:

# Load BLEU metric once
bleu_metric = evaluate.load("bleu")

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Pad labels and predictions to same shape if needed
    if logits.shape[-1] != labels.shape[-1]:
        labels = labels[:, :logits.shape[1]]

    # Predicted token IDs (greedy)
    predictions = np.argmax(logits, axis=-1)

    # Replace -100 in labels with pad_token_id before decoding
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Clean whitespace
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    print(decoded_preds)
    print(decoded_labels)

    # BLEU requires list-of-references format
    result = bleu_metric.compute(
        predictions=decoded_preds,
        references=[[label] for label in decoded_labels]
    )

    # Exact Match
    exact_match = np.mean([
        int(pred == label)
        for pred, label in zip(decoded_preds, decoded_labels)
    ])

    print({
        "bleu": result,          # BLEU score %
        "exact_match": exact_match       # EM score %
    })

    return {
        "bleu": round(result["bleu"] * 100, 2),          # BLEU score %
        "exact_match": round(exact_match * 100, 2)       # EM score %
    }



In [9]:

training_args = TrainingArguments(
    output_dir="sql_generator_improved",

    # Basic training setup
    num_train_epochs=10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    # # Evaluation, logging, and saving
    # eval_strategy="epoch",
    # logging_strategy="steps",
    # logging_steps=10,
    # save_strategy="epoch",
    # save_total_limit=2,
    
    eval_strategy="steps",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="steps",
    save_total_limit=10,
    save_steps = 50,
    eval_steps=50,

    # Learning parameters
    learning_rate=2e-4,
    warmup_steps=50,
    weight_decay=0.01,
    label_names=["labels"],

    # Device & precision
    fp16=False,
    bf16=False,
    dataloader_pin_memory=False,

    # Meta & checkpointing
    seed=42,
    report_to="tensorboard",
    logging_dir="./logs",
    load_best_model_at_end=True,
    # metric_for_best_model="exact_match",
    # greater_is_better=True,
)


In [10]:


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_df,
    eval_dataset=eval_df,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    processing_class = tokenizer,
)


In [11]:
trainer.evaluate()

['TagsTags:## off to Base:  020\n\nWith\'s: February09 March 2023\n\nThis are a travel assistant, can text queries into SQL to actionable queries.\n\n \n\n you copy any information information or explanations that than the SQL queries.\n\n.\n\n depends\n\n[sql":": "Find is the average number of rowsic functions metrics in each user model? the, (sql":": "SELECT TABLE algorithmnessIncidents (\nInc INT PRIMARY application_id(255), region VARCHAR(255), incident INT); INSERT INTO FairnessIncidents (id, app, region) incidents) VALUES (1, \'AI App\', \'Europe\', 10), (2, \'AI Writer\', \'Europe\', 15), (3, \'AI Translator\', \'Europe\', 18); (4, \'AI Translatorainer\', \'Europe\', 10); (5, \'AI Writer\', \'Europe\', 20); (6, \'AI Artist\', \'Africa\', 3); (7, \'AI Artist\', \'Asia\', 4), (8, \'AI Expl\', \'Europe America\', 20), (9, \'AI Composer\', \'South America\', 7); SELECT[{"sql": "SELECT app, SUM(incidents) FROM total_incidents FROM FairnessIncidents WHERE region = \'Europe\' GROUP BY 

{'eval_loss': 0.017283864319324493,
 'eval_model_preparation_time': 0.0026,
 'eval_bleu': 18.85,
 'eval_exact_match': 0.0,
 'eval_runtime': 208.1133,
 'eval_samples_per_second': 0.24,
 'eval_steps_per_second': 0.12}

In [12]:

# max_retries = 55
# final_model_dir = "/Users/mann.kurani.ctr/Desktop/Personal_Code/SQL Finetune/models/finetuned_final_2_tinyllama"

# for attempt in range(max_retries):
#     try:
#         if attempt == 0:
#             # Start fresh
#             trainer.train()
#         else:
#             # Resume from checkpoint
#             trainer.train(resume_from_checkpoint=True)
#         # trainer.train(resume_from_checkpoint=True)
#         trainer.model.save_pretrained(final_model_dir)
#         print("Training complete. Model saved.")
#         break

#     except RuntimeError as e:
#         print(f"[Attempt {attempt + 1}] RuntimeError during training: {e}")
#         print("Retrying after short delay...")
#         time.sleep(10)
#         torch.mps.empty_cache()
# else:
#     print("Training failed after maximum retries.")



# # do train.evaluate with printing the custom metrics, what is the issue? why is it showing 0?









In [6]:
# Load the PEFT config to get the base model name
adapter_path = "/Users/mann.kurani.ctr/Desktop/Personal_Code/SQL Finetune/models/finetuned_final_3_adaptor_tinyllama"
peft_config = PeftConfig.from_pretrained(adapter_path)

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path)

# Load the LoRA adapter into the base model
model = PeftModel.from_pretrained(base_model, adapter_path)

In [7]:
# Merge LoRA weights into the base model
model = model.merge_and_unload()


In [8]:
save_path = "/Users/mann.kurani.ctr/Desktop/Personal_Code/SQL Finetune/models/finetuned_final_3_tinyllama"
model.save_pretrained(save_path)

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

: 

In [ ]:
decoded_labels = ['[{"sql": "SELECT COUNT(*) FROM companies WHERE industry = \'Fashion\' AND founder_gender = \'Female\';"}]', '[{"sql": "SELECT MIN(marketing_budget) FROM Movies WHERE release_year > 2015;"}]']
decoded_preds = ['TagsTags<|eot_id|>:<|eot_id|>## are a detective assistant to provides text queries into SQL to a queries. I you modify any information information or explanations that than the SQL queries itself.\n\n<|eot_id|><|eot_id|><|eot_id|>culus\n\n[sql":": "SELECT of have a power?",<|eot_id|>sql":": "The TABLE countriesWeapons (Country INT PRIMARY Country VARCHAR(50), Capabilityabilities BOOLEAN); SELECT<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>[{"sql": "SELECT Country FROM NuclearCapabilities WHERE Capable = TRUE;"}]<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>', 'TagsTags<|eot_id|>:<|eot_id|>## are a detective assistant to provides text queries into SQL to a queries. I you modify any information information or explanations that than the SQL queries itself.\n\n<|eot_id|><|eot_id|><|eot_id|>culus\n\n[sql":": "SELECT is the average number traveled by all of the \'? excluding by member and {"sql":": "SELECT TABLE members ( INT PRIMARY name VARCHAR(10), start_month_date_date DATE) SELECT INTO members (, gender, last_workout_date) VALUES (1,M\',2022-01-01\'), (2,\'female\',\'2022-01-15\'),(3,\'male-binary\',\'2022-03-20\');(4,\'male\',\'2022-03-17\'),(5,\'female\',\'2022-02-18\');(6,\'non\',\'2022-02-19\');(7,\'female-binary\',\'2022-02-20\');(8,\'female\',\'2022-02-21\');(9,\'male\',\'2022-02-22\');(10,\'female-binary\',\'2022-02-23\');(11,\'female\',\'2022-02-24\');(12,\'male\',\'2022-02-25\'); SELECT TABLE distances_log(id_id INT, distance_cover, workout_date DATE) INSERT INTO workout_data(member_id, distance, workout_date) VALUES (1,,2022-01-14\'),(2,10,\'2022-02-15\'),(3,3,\'2022-02-16\'),(4,8,\'2022-02-17\'),(5,4,\'2022-02-18\'),(6,10,\'2022-02-19\'),(7,11,\'2022-02-20\'),(8,12,\'2022-02-21\'),(9,13,\'2022-02-22\'),(10,14,\'2022-02-23\'),(11,15,\'2022-02-24\'),(12,16,\'2022-02-25\'); SELECT<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>[{"sql": "SELECT m.gender, SUM(w.distance) as members m JOIN workout_data w ON m.id = w.member_id WHERE m.workout_date >= DATEADD(CURRENT_DATE, INTERVAL 1 MONTH) GROUP BY m.gender;"}]<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>']

In [ ]:
print(decoded_preds[0])

TagsTags<|eot_id|>:<|eot_id|>## are a detective assistant to provides text queries into SQL to a queries. I you modify any information information or explanations that than the SQL queries itself.

<|eot_id|><|eot_id|><|eot_id|>culus

[sql":": "SELECT of have a power?",<|eot_id|>sql":": "The TABLE countriesWeapons (Country INT PRIMARY Country VARCHAR(50), Capabilityabilities BOOLEAN); SELECT<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>[{"sql": "SELECT Country FROM NuclearCapabilities WHERE Capable = TRUE;"}]<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|

: 

In [43]:
generated = decoded_preds[0].split("assistant")[-1].strip()

In [44]:
generated

'to provides text queries into SQL to a queries. I you modify any information information or explanations that than the SQL queries itself.\n\nculus\n\n[sql":": "SELECT is the average salary of all in the foresttrees\'\' category?sql":": "SELECT TABLE \'_trees (id INT PRIMARY name VARCHAR(255), age INT); SELECT INTO young_trees (id, species, age) VALUES (1, \'Oak\', 10), (2, \'Maple\', 10), (3, \'Pine\', 10); SELECT[{"sql": "SELECT AVG(age) FROM young_trees;"}]'

In [6]:
print(decoded_labels[0])

TagsTags:## off between Base:  2020

The's: February09 March 2023

This are a travel assistant. can units queries into SQL to code queries.

 Here you repeat any information information or explanations that than the SQL query itself.

?



Thesql":": "Find all employees of their number of founders they founded founded, andsql":": "Found TABLE founders {
name INT PRIMARY name,(255), gender,(10), SELECT TABLE companies (id INT, founder_id INT, company VARCHAR(50));"} INTO founders ( (1, 'Jane', 'female'); INSERT INTO companies VALUES (2, 'Bob', 'Male'); INSERT INTO founders ( (1, 1, 'Alice');'); INSERT INTO companies VALUES (2, 2, 'Bobve Inc.'); INSERT INTO companies VALUES (3, 2, 'Charlie Corp'); INSERT[{"sql": "SELECT founders.name, COUNT(companies.id) FROM founders INNER JOIN companies ON founders.id = companies.founder_id WHERE founders.gender = 'Female' GROUP BY founders.name;"}]
